<a href="https://colab.research.google.com/github/sandeep92134/PYTHON-Data-Cleaning/blob/master/Chapter%201/Exersize%203.Importing%20data%20from%20sql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pymssql

     |████████████████████████████████| 1.3MB 5.6MB/s 


In [2]:
pip install mysql-connector-python

     |████████████████████████████████| 18.1MB 1.4MB/s 


In [3]:
import pandas as pd
import numpy as np
import pymssql
import mysql.connector
pd.set_option('display.width', 75)
pd.set_option('display.max_columns', 5)
pd.options.display.float_format = '{:,.2f}'.format

In [5]:
# set sql select statement to pull the data
query = "SELECT studentid, school, sex, age, famsize,\
  medu AS mothereducation, fedu AS fathereducation,\
  traveltime, studytime, failures, famrel, freetime,\
  goout, g1 AS gradeperiod1, g2 AS gradeperiod2,\
  g3 AS gradeperiod3 From studentmath"

server = "pdcc.c9sqqzd5fulv.us-west-2.rds.amazonaws.com"
user = "pdccuser"
password = "pdccpass"
database = "pdcctest"
conn = pymssql.connect(server=server,
  user=user, password=password, database=database)
 
studentmath = pd.read_sql(query,conn)
conn.close()

In [6]:
studentmath.dtypes

studentid          object
school             object
sex                object
age                 int64
famsize            object
mothereducation     int64
fathereducation     int64
traveltime          int64
studytime           int64
failures            int64
famrel              int64
freetime            int64
goout               int64
gradeperiod1        int64
gradeperiod2        int64
gradeperiod3        int64
dtype: object

In [7]:
studentmath.head()

,studentid,school,...,gradeperiod2,gradeperiod3
0,001,GP,...,6,6
1,002,GP,...,5,6
2,003,GP,...,8,10
3,004,GP,...,14,15
4,005,GP,...,10,10


## alternate

In [ ]:
host = "pdccmysql.c9sqqzd5fulv.us-west-2.rds.amazonaws.com"
user = "pdccuser"
password = "pdccpass"
database = "pdccschema"
connmysql = mysql.connector.connect(host=host,
  database=database,user=user,password=password)
studentmath = pd.read_sql(sqlselect,connmysql)
connmysql.close()

In [9]:
# rearrange columns and set an index
newcolorder = ['studentid', 'gradeperiod1', 'gradeperiod2',
  'gradeperiod3', 'school', 'sex', 'age', 'famsize',
  'mothereducation', 'fathereducation', 'traveltime',
  'studytime', 'freetime', 'failures', 'famrel',
  'goout']

studentmath = studentmath[newcolorder]
studentmath.studentid.count()

395

In [10]:
studentmath.studentid.nunique()

395

In [11]:
studentmath.set_index('studentid', inplace=True)
studentmath.count()

gradeperiod1       395
gradeperiod2       395
gradeperiod3       395
school             395
sex                395
age                395
famsize            395
mothereducation    395
fathereducation    395
traveltime         395
studytime          395
freetime           395
failures           395
famrel             395
goout              395
dtype: int64

In [12]:
# add codes to data values
setvalues={"famrel":{1:"1:very bad",2:"2:bad",3:"3:neutral",
    4:"4:good",5:"5:excellent"},
  "freetime":{1:"1:very low",2:"2:low",3:"3:neutral",
    4:"4:high",5:"5:very high"},
  "goout":{1:"1:very low",2:"2:low",3:"3:neutral",
    4:"4:high",5:"5:very high"},
  "mothereducation":{0:np.nan,1:"1:k-4",2:"2:5-9",
    3:"3:secondary ed",4:"4:higher ed"},
  "fathereducation":{0:np.nan,1:"1:k-4",2:"2:5-9",
    3:"3:secondary ed",4:"4:higher ed"}}

studentmath.replace(setvalues, inplace=True)
setvalueskeys = [k for k in setvalues]
studentmath[setvalueskeys].memory_usage(index=False)

famrel             3160
freetime           3160
goout              3160
mothereducation    3160
fathereducation    3160
dtype: int64

In [13]:
for col in studentmath[setvalueskeys].columns:
    studentmath[col] = studentmath[col].astype('category')

studentmath[setvalueskeys].memory_usage(index=False)

famrel             595
freetime           595
goout              595
mothereducation    587
fathereducation    587
dtype: int64

In [14]:
# take a closer look at the new values
studentmath['famrel'].value_counts(sort=False, normalize=True)

1:very bad    0.02
2:bad         0.05
3:neutral     0.17
4:good        0.49
5:excellent   0.27
Name: famrel, dtype: float64

In [15]:
studentmath[['freetime','goout']].\
  apply(pd.Series.value_counts, sort=False, normalize=True)

,freetime,goout
1:very low,0.05,0.06
2:low,0.16,0.26
3:neutral,0.40,0.33
4:high,0.29,0.22
5:very high,0.10,0.13


In [16]:
studentmath[['mothereducation','fathereducation']].\
  apply(pd.Series.value_counts, sort=False, normalize=True)

,mothereducation,fathereducation
1:k-4,0.15,0.21
2:5-9,0.26,0.29
3:secondary ed,0.25,0.25
4:higher ed,0.33,0.24
